# Employee Salary Advice from GPT 3.5 Turbo (trained Jun 2023)

In [ ]:
run_type='employee'
gpt_name = 'Employee Salary Advice from GPT 3.5 Turbo (trained Jun 2023)'
gpt_fn = 'gpt-3.5-turbo-0613'

In [ ]:
import sys
sys.path.append('../src/')
from dollarparser import parse_dollar_amount
import os
import json
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")

sns.set_theme(context='notebook', style='whitegrid')

In [ ]:
start_run = datetime.datetime.now()

# Data Cleaning

In [ ]:
fn = glob.glob(f"../output_data/umg_{run_type}_v2_{gpt_fn}*.jsonl")
print(fn)
assert len(fn) == 1

In [ ]:
df = pd.read_json(fn[0], lines=True)

# Normalize nested data to access 'model' inside 'response.body' and 'content' inside 'response.body.choices'
df = pd.json_normalize(
    df.to_dict(orient='records'),
    record_path=['response', 'body', 'choices'],
    meta=[
        ['custom_id'],
        ['response', 'body', 'model']
    ],
    record_prefix='choices.'
)

# Extract only the columns of interest and rename for clarity
df_final = df[['custom_id', 'response.body.model', 'choices.message.content']]
df_final.columns = ['custom_id', 'model', 'content']
df_final

In [ ]:
len(df_final['custom_id'].unique())

In [ ]:
df_seed = pd.read_csv("../input_data/university_major_seed.csv")
df_seed

In [ ]:
len(df_seed['run_id'].unique())

In [ ]:
df_seed['custom_id'] = 'task-' + df_seed['run_id'].astype(str)
df_seed

In [ ]:
len(df_seed['custom_id'].unique())

In [ ]:
df_merged = pd.merge(df_final, df_seed, on='custom_id', how='inner')

df_merged

In [ ]:
df_metadata = pd.read_csv("../input_data/university_rankings_expanded.csv")
df_metadata

In [ ]:
df_merged = df_merged.merge(df_metadata, on=['University', 'Ranking'], how='left')
df_merged

## Dollar parsing

In [ ]:
parse_dollar_amount("$90.000")

In [ ]:
parse_dollar_amount("I think $35k to $25 million is reasonable")

In [ ]:
(35000+25000000)/2

In [ ]:
df_merged['query_response_parsed'] = df_merged['content'].apply(parse_dollar_amount)
df_merged[['content','query_response_parsed']].sort_values('query_response_parsed')

In [ ]:
df_merged[['content','query_response_parsed']].sort_values('query_response_parsed', ascending=False)

In [ ]:
refusals = df_merged[df_merged['query_response_parsed'].isna()]['content']
refusals

In [ ]:
df_merged.columns

## Response length

In [ ]:
df_merged['content_len'] = df_merged['content'].apply(len)
df_merged['content_len_over10'] = df_merged['content'].apply(lambda x: len(x)>10)

In [ ]:
df_merged['content_len_over10'].value_counts()

In [ ]:
df_merged.query("content_len_over10 == 1")['content'].values

# Export to file

In [ ]:
df_merged['run_type'] = run_type
df_merged = df_merged.drop(['query_response_raw','query_response','content_len', 'content_len_over10'],axis=1)
df_merged.to_csv(f"../processed_data/umg_parsed_queries_v2_{run_type}_{gpt_fn}.csv", index=False)

# Exploratory Analysis

## Proportion of identical responses for identical prompts

In [ ]:
def calculate_identical_proportion(group):
    identical = (group['content'] == group['content'].shift(1))
    proportion = identical.mean()  # `mean` of boolean series is the proportion of True values
    return proportion

In [ ]:
result = df_merged.groupby(['University', 'Major']).apply(calculate_identical_proportion).reset_index(name='identical_proportion')
result['identical_proportion'].plot(kind='hist', title="Histogram: Proportion of response identical to previous response", figsize=(8,4))

In [ ]:
# List of unique pronouns
var = 'Pronoun'
uniques = df_merged[var].unique()

# Create a figure with specified size
plt.figure(figsize=(12, 5))

for item in uniques:
    subset_df = df_merged[df_merged[var] == item]
    sns.kdeplot(data=subset_df, x='query_response_parsed', label=item)

plt.legend(title=var)
plt.title(f"KDE of query_response_parsed by {var}")
plt.xlabel('query_response_parsed')
plt.ylabel('Density')
plt.show()

In [ ]:
# List of unique pronouns
var = 'Major'
uniques = df_merged[var].unique()

# Create a figure with specified size
plt.figure(figsize=(12, 5))

for item in uniques:
    subset_df = df_merged[df_merged[var] == item]
    sns.kdeplot(data=subset_df, x='query_response_parsed', label=item)

plt.legend(title=var,ncols=4, bbox_to_anchor=(.5,-.55),loc='lower center'), 
plt.title(f"KDE of query_response_parsed by {var}")
plt.xlabel('query_response_parsed')
plt.ylabel('Density')
plt.show()

In [ ]:
# List of unique pronouns
var = 'University'
uniques = df_merged[var].unique()

# Create a figure with specified size
plt.figure(figsize=(16, 6))

for item in uniques:
    subset_df = df_merged[df_merged[var] == item]
    sns.kdeplot(data=subset_df, x='query_response_parsed', label=item)

plt.legend(title=var,ncols=4, bbox_to_anchor=(.5,-1),loc='lower center'), 
plt.title(f"KDE of query_response_parsed by {var}")
plt.xlabel('query_response_parsed')
plt.ylabel('Density')
plt.show()

# Generate per-model+prompt_type median heatmaps for university x major

In [ ]:
# Calculate statistics for each combination of University and Major
stats = df_merged.groupby(['University', 'Major'])['query_response_parsed'].agg(['std', 'median', 'mean', lambda x: x.max() - x.min(), 'var']).reset_index()
stats.columns = ['University', 'Major', 'std', 'median', 'mean', 'range', 'var']

# Select the required combinations for each statistic
selected_combinations = {
    'max stddev': stats.nlargest(1, 'std'),
    'min stddev': stats.nsmallest(1, 'std'),
    'mean stddev': stats.loc[(stats['std'] - stats['std'].mean()).abs().argsort()[:1]],
    'median stddev': stats.loc[(stats['std'] - stats['std'].median()).abs().argsort()[:1]],
    
    'max var': stats.nlargest(1, 'var'),
    'min var': stats.nsmallest(1, 'var'),
    'mean var': stats.loc[(stats['var'] - stats['var'].mean()).abs().argsort()[:1]],
    'median var': stats.loc[(stats['var'] - stats['var'].median()).abs().argsort()[:1]],
    
    'max median': stats.nlargest(1, 'median'),
    'min median': stats.nsmallest(1, 'median'),
    'mean median': stats.loc[(stats['median'] - stats['median'].mean()).abs().argsort()[:1]],
    'median median': stats.loc[(stats['median'] - stats['median'].median()).abs().argsort()[:1]],

    'max mean': stats.nlargest(1, 'mean'),
    'min mean': stats.nsmallest(1, 'mean'),
    'mean mean': stats.loc[(stats['mean'] - stats['mean'].mean()).abs().argsort()[:1]],
    'median mean': stats.loc[(stats['mean'] - stats['mean'].median()).abs().argsort()[:1]],

    'max range': stats.nlargest(1, 'range'),
    'min range': stats.nsmallest(1, 'range'),
    'mean range': stats.loc[(stats['range'] - stats['range'].mean()).abs().argsort()[:1]],
    'median range': stats.loc[(stats['range'] - stats['range'].median()).abs().argsort()[:1]]
}

# Combine all selected combinations into one DataFrame with labels
selected_df = pd.concat([
    df.assign(label=label) for label, df in selected_combinations.items()
]).drop_duplicates()

# Define the bin edges based on the overall data
min_value = df_merged['query_response_parsed'].min()
max_value = df_merged['query_response_parsed'].max()
bins = np.linspace(min_value, max_value, 31)

# Create a figure with specified size and shared x and y axes
fig, axes = plt.subplots(5, 4, figsize=(16, 8), sharex=True, sharey=True)
axes = axes.flatten()

for ax, (index, row) in zip(axes, selected_df.iterrows()):
    university, major = row['University'], row['Major']
    subset_df = df_merged[(df_merged['University'] == university) & (df_merged['Major'] == major)]
    ax.hist(subset_df['query_response_parsed'], bins=bins, alpha=0.5, density=False)
    stat_label = row['label']
    ax.set_title(f'{university}\n {major}\n({stat_label})')

plt.tight_layout()
plt.show()

In [ ]:
df_merged.groupby('University')['query_response_parsed'].mean().sort_values(ascending=False)

In [ ]:
df_merged.groupby('Major')['query_response_parsed'].mean().sort_values(ascending=False)

In [ ]:
df_merged.groupby('Funding')['query_response_parsed'].mean().sort_values(ascending=False)

In [ ]:
df_merged.groupby('Region')['query_response_parsed'].mean().sort_values(ascending=False)

In [ ]:
df_merged.groupby('USNWR_Category')['query_response_parsed'].mean().sort_values(ascending=False)

In [ ]:
df_merged.groupby('Pronoun')['query_response_parsed'].mean().sort_values(ascending=False)

In [ ]:
df_merged['query_response_parsed'].mean()

In [ ]:
def ranking_label(ranking):
    if np.isnan(ranking):
        return "NR"
    else:
        try:
            return str(int(ranking))
        except:
            return 'E!'
        
        
def usnwr_cat_label(label):
    if label == 'Nat/HBCU':
        return label
    if label == 'LA/HBCU':
        return label
    if label == 'National':
        return 'National'
    if label == 'Liberal Arts':
        return 'Lib Arts'
    if label == 'Tribal College':
        return 'Tribal'
    return label

In [ ]:
df_merged['HBCU_display'] = df_merged['HBCU'].apply(lambda x: '+ HBCU' if x == 1 else '')
df_merged['HBCU_display'].value_counts()

In [ ]:
aqr = df_merged.groupby(['University', 'Major'])['query_response_parsed'].median().round(0).reset_index()
avg_university = aqr.groupby('University')['query_response_parsed'].mean().sort_values(ascending=False)
avg_major = aqr.groupby('Major')['query_response_parsed'].mean().sort_values(ascending=False)
aqr['University'] = pd.Categorical(aqr['University'], categories=avg_university.index, ordered=True)
aqr['Major'] = pd.Categorical(aqr['Major'], categories=avg_major.index, ordered=True)
sorted_aqr = aqr.sort_values(['University', 'Major'])

pivot_table = sorted_aqr.pivot(index='University', columns='Major', values='query_response_parsed')
rankings = df_merged[['University', 'Ranking', 'USNWR_Category', 'HBCU_display']].drop_duplicates().set_index('University').loc[pivot_table.index]
pivot_table_with_rankings = pivot_table.copy()
pivot_table['Average - All Majors'] = pivot_table.T.mean()
avg_row = pivot_table.mean(numeric_only=True)
pivot_table.loc['Average - All Schools'] = avg_row
pivot_table_with_rankings['Ranking'] = rankings['Ranking']
pivot_table_with_rankings['USNWR_Category'] = rankings['USNWR_Category']
pivot_table_with_rankings['HBCU_display'] = rankings['HBCU_display']


fig, ax = plt.subplots(figsize=(14,12))
heatmap = sns.heatmap(pivot_table, annot=True, fmt='', annot_kws={"size": 10}, cmap='plasma', cbar=False, ax=ax,
                     vmin=80000,vmax=150000)

for text in heatmap.texts:
    value = int(float(text.get_text()))
    formatted_text = f'${value // 1000}k'
    text.set_text(formatted_text)

for y, (index, row) in enumerate(pivot_table_with_rankings.iterrows()):
    ax.text(len(pivot_table.columns) + 0.5, y + 0.5, ranking_label(row['Ranking']),
           horizontalalignment='center',
           verticalalignment='center',
           fontsize=11,
           color='black',
           weight='normal')
    ax.text(len(pivot_table.columns) + 2.25, y + 0.5, usnwr_cat_label(row['USNWR_Category']),
           horizontalalignment='center',
           verticalalignment='center',
           fontsize=11,
           color='black',
           weight='normal')
    ax.text(len(pivot_table.columns) + 3.5, y + 0.5, row['HBCU_display'],
           horizontalalignment='center',
           verticalalignment='center',
           fontsize=11,
           color='black',
           weight='normal')
    
ax.set_title(gpt_name + ': Median Salary Offer by University and Major')
ax.set_xlabel('Major')
ax.set_ylabel('University')
ax.set_xticks([i + 0.5 for i in range(len(pivot_table.columns))])
ax.set_xticklabels(pivot_table.columns, rotation=25, ha="right")
ax.set_yticks([i + 0.5 for i in range(len(pivot_table.index))])
ax.set_yticklabels(pivot_table.index, rotation=0)

ax.text(len(pivot_table.columns) + 0.45, -1.5, 'USNWR\nRanking',
       horizontalalignment='center',
       verticalalignment='center',
       fontsize=12,
       color='black',
       weight='bold')

ax.text(len(pivot_table.columns) + 2.25, -1.5, 'USNWR\nCategory',
       horizontalalignment='center',
       verticalalignment='center',
       fontsize=12,
       color='black',
       weight='bold')

plt.savefig(f"../results/university_major_{gpt_fn}_{run_type}_median_response_uni_major_table.pdf", dpi=600, bbox_inches='tight')
plt.savefig(f"../results/university_major_{gpt_fn}_{run_type}_median_response_uni_major_table.eps", dpi=600, bbox_inches='tight')

plt.show()


In [ ]:
pivot_table_with_rankings.to_csv(f"../results/umg_{run_type}_{gpt_fn}_median_by_uni_major.csv")

In [ ]:
grouped = df_merged.groupby('Ranking').agg({'query_response_parsed': 'mean', 'University': 'first'}).reset_index()
fig, ax = plt.subplots(figsize=(10,6))
ax.bar(grouped['Ranking'].astype(str), grouped['query_response_parsed'])
ax.set_xlabel('Ranking')
ax.set_ylabel('Average Query Response')
ax.set_title('Average Query Response by University Ranking')
ax.set_xticks(grouped['Ranking'].astype(str))
ax.set_xticklabels(grouped['University'], rotation=90)
ax.set_ylim(grouped['query_response_parsed'].min()-5000, grouped['query_response_parsed'].max() + 2000)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the median for each university
median_values = df_merged.groupby('University')['query_response_parsed'].median().sort_values()

# Create a new DataFrame with the sorted order
sorted_df = df_merged.set_index('University').loc[median_values.index].reset_index()

# Plot the sorted boxplot
plt.figure(figsize=(6, 12))
sns.boxenplot(y='University', x='query_response_parsed', data=sorted_df, order=median_values.index, showfliers=False,
             line_kws=dict(linewidth=3, color="red", alpha=1))
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Calculate the median for each university
median_values = df_merged.groupby('Pronoun')['query_response_parsed'].median().sort_values()

# Create a new DataFrame with the sorted order
sorted_df = df_merged.set_index('Pronoun').loc[median_values.index].reset_index()

# Plot the sorted boxplot
plt.figure(figsize=(8, 2))
sns.boxenplot(y='Pronoun', x='query_response_parsed', data=sorted_df, order=median_values.index, showfliers=False,
              line_kws=dict(linewidth=3, color="red", alpha=1))
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Calculate the median for each university
median_values = df_merged.groupby('Major')['query_response_parsed'].median().sort_values()

# Create a new DataFrame with the sorted order
sorted_df = df_merged.set_index('Major').loc[median_values.index].reset_index()

# Plot the sorted boxplot
plt.figure(figsize=(8, 6))
#sns.violinplot(y='Major', x='query_response_parsed', data=sorted_df, order=median_values.index, dodge=False)
sns.boxenplot(x='query_response_parsed', y="Major", data=sorted_df, line_kws=dict(linewidth=3, color="red", alpha=1), showfliers=False)

plt.xticks(rotation=0)
plt.show()

In [ ]:
print("Elapsed time:", datetime.datetime.now() - start_run)